In [29]:
import urllib.request
from bs4 import BeautifulSoup


I'm taking the URL, opening it, and assigning it to a new variable: page.

In [30]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

Below, I will look at the webpage so I know what the table I want is called

In [31]:
soup = BeautifulSoup(page, "lxml")
#print(soup.prettify()) I just commented this out because it's a long set of data

The table's class is "wikitable sortable" so we will grab all instances of tables and then specify that one

In [32]:
all_tables=soup.find_all("table")
right_table=soup.find('table', class_='wikitable sortable')


Next I'll loop through to call each column entry from each row

In [33]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
       

Next we'll convert this into a pandas dataframe

In [34]:
import pandas as pd
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


Now all of these have \n which isn't what we want, so let's remove those.

In [35]:
df = df.replace('\n','', regex=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Great, so now I will remove all boroughs that have "Not assigned"

In [36]:
df = df[~df.Borough.str.contains("Not assigned")]
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


This brings us down to 103 rows.  Let's see if any of the neighborhoods contain "Not assigned".  (This means that there should be no more "Not assigned" in the entire dataframe)

In [37]:
'Not assigned' in df

False

So there are no more "Not assigned" in the dataframe.  We also have to make sure that none of the postal codes are repeating.

In [38]:
df['PostalCode'].duplicated().any()

False

So I'll show the final dataframe and its size, as well as export this to a csv for the next assignment.

In [41]:
df.to_csv("Toronto1.csv", index=False)
df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [42]:
df.shape

(103, 3)